# 淘宝用户行为分析

## 1.数据描述
### 1.1概述
[数据来源](https://tianchi.aliyun.com/dataset/dataDetail?dataId=649&userId=1)
UserBehavior是阿里巴巴提供的一个淘宝用户行为数据集，用于隐式反馈推荐问题的研究。
### 1.2介绍
| 文件名称     | 说明                 | 包含特征                                 |  
| ---------------- | ---------------------- | -------------------------------------------- |  
| UserBehavior.csv | 包含所有的用户行为数据 | 用户ID，商品ID，商品类目ID，行为类型，时间戳 |  

UserBehavior.csv
本数据集包含了2017年11月25日至2017年12月3日之间，有行为的约一百万随机用户的所有行为（行为包括点击、购买、加购、喜欢）。数据集共3.4G。数据集的组织形式和MovieLens-20M类似，即数据集的每一行表示一条用户行为，由用户ID、商品ID、商品类目ID、行为类型和时间戳组成，并以逗号分隔。关于数据集中每一列的详细描述如下：


| 列名称     | 说明                                               |
| :--------- | :------------------------------------------------- |
| 用户ID     | 整数类型，序列化后的用户ID                         |
| 商品ID     | 整数类型，序列化后的商品ID                         |
| 商品类目ID | 整数类型，序列化后的商品所属类目ID                 |
| 行为类型   | 字符串，枚举类型，包括('pv', 'buy', 'cart', 'fav') |
| 时间戳     | 行为发生的时间戳                                   |

注意到，用户行为类型共有四种，它们分别是

| 行为类型 | 说明                     |
| :------- | :----------------------- |
| pv       | 商品详情页pv，等价于点击 |
| buy      | 商品购买                 |
| cart     | 将商品加入购物车         |
| fav      | 收藏商品                 |

关于数据集大小的一些说明如下

| 维度         | 数量        |
| :----------- | :---------- |
| 用户数量     | 987,994     |
| 商品数量     | 4,162,024   |
| 用户数量     | 987,994     |
| 商品类目数量 | 9,439       |
| 所有行为数量 | 100,150,807 |


## 2.数据分析
### 2.1 描述性分析
导入项目需要工具包


In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import os

读取数据

In [2]:
def list_dir_files(root_dir, ext=None):
    """
    列出文件夹中的文件, 深度遍历
    :param root_dir: 根目录
    :param ext: 后缀名
    :return: [文件路径列表, 文件名称列表]
    """
    names_list = []
    paths_list = []
    for parent, _, fileNames in os.walk(root_dir):
        for name in fileNames:
            if name.startswith('.'):  # 去除隐藏文件
                continue
            if ext:  # 根据后缀名搜索
                if name.endswith(tuple(ext)):
                    names_list.append(name)
                    paths_list.append(os.path.join(parent, name))
            else:
                names_list.append(name)
                paths_list.append(os.path.join(parent, name))
    #paths_list, names_list = sort_two_list(paths_list, names_list)
    #return paths_list, names_list
    return paths_list,names_list

#1.读取数据

Raw_data_df = pd.DataFrame(columns = ['用户ID', '商品ID','商品类目ID','行为类型','时间戳'])

for path in list_dir_files(r'D:\github\数据集\淘宝用户行为数据分析',"csv")[0]:
    try:
        df = pd.read_csv(path, names=['用户ID', '商品ID','商品类目ID','行为类型','时间戳'])
        df = df.loc[(df['时间戳'] > 1511539200 ) & (df['时间戳'] <= 1512316799)]       
        df = df.sample(frac=0.1, replace=True, random_state=1)
        Raw_data_df = Raw_data_df.append(df)
    except :
        print("load file error")


描述性分析

In [3]:
#查看数据基本信息
print(Raw_data_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10009509 entries, 4324551 to 4360514
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   用户ID    object
 1   商品ID    object
 2   商品类目ID  object
 3   行为类型    object
 4   时间戳     object
dtypes: object(5)
memory usage: 458.2+ MB
None


In [37]:
#时间戳转化为时间,并只保留日期，方便之后的分析
#Raw_data_df['time'] = pd.to_datetime(Raw_data_df['时间戳'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
Raw_data_df['时间'] = pd.to_datetime(Raw_data_df['时间戳'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai').dt.date.astype(str)
Raw_data_df['时段'] = pd.to_datetime(Raw_data_df['时间戳'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai').dt.hour.astype(int)

In [38]:
#预览数据
pprint(Raw_data_df.head(20))

            用户ID     商品ID   商品类目ID 行为类型         时间戳          时间  时段  \
4324551   278535  2689159  4238676   pv  1511783167  2017-11-27  19   
4688525   295426  4617533  1879194   pv  1512203473  2017-12-02  16   
3939379   260542  4168202  1320293   pv  1511789350  2017-11-27  21   
5033448   311267  4597853   411153   pv  1512126535  2017-12-01  19   
4269849   276048  2985248  2520377   pv  1512230078  2017-12-02  23   
2327798   188541  1100861  1320293   pv  1511959363  2017-11-29  20   
414006    102632  1478936  2520377  buy  1512173316  2017-12-02   8   
3065273   221096  3226795  4756105   pv  1512290464  2017-12-03  16   
5122265   315247  3285945   490076   pv  1512200477  2017-12-02  15   
1216476   138390  3362170  4022701   pv  1512228183  2017-12-02  23   
2611710   200886   491998  1813610   pv  1512020276  2017-11-30  13   
5267418   322021   468283  3004853   pv  1512000112  2017-11-30   8   
3323852   232801  4803192  2096639   pv  1511881810  2017-11-28  23   
477276

In [39]:
#查看是否有缺失值
print(Raw_data_df.isnull().sum())

用户ID      0
商品ID      0
商品类目ID    0
行为类型      0
时间戳       0
时间        0
时段        0
time      0
dtype: int64


数据没有缺失，很完整。

In [40]:
#查看用户行为是否符合规定：即只有PV，buy，cart，fav 四类
print(Raw_data_df['行为类型'].unique())

['pv' 'buy' 'fav' 'cart']


用户行为符合规定，即只有规定的4个指标。

In [41]:
#查看原始数据时间跨度
print(Raw_data_df['时间'].min(),Raw_data_df['时间'].max())

2017-11-26 2017-12-03


时间跨度从1919-09-19 到 2037-04-09，有超出分析时间范围的数据，要清洗掉

### 2.2数据清洗

我们要分析的范围是2017-11-25到2017-12-03,接下来清洗掉不需要分析的数据。

In [42]:
Raw_data_df=Raw_data_df.loc[(Raw_data_df['时间'] > '2017-11-25') & (Raw_data_df['时间'] <= '2017-12-03')]
print(Raw_data_df)

           用户ID     商品ID   商品类目ID  行为类型         时间戳          时间  时段  \
4324551  278535  2689159  4238676    pv  1511783167  2017-11-27  19   
4688525  295426  4617533  1879194    pv  1512203473  2017-12-02  16   
3939379  260542  4168202  1320293    pv  1511789350  2017-11-27  21   
5033448  311267  4597853   411153    pv  1512126535  2017-12-01  19   
4269849  276048  2985248  2520377    pv  1512230078  2017-12-02  23   
...         ...      ...      ...   ...         ...         ...  ..   
1414936  512143  2625832  2702525  cart  1511673050  2017-11-26  13   
1928182   53688  2567622   570735    pv  1512286820  2017-12-03  15   
1273657  505824  2685312   383749    pv  1512179930  2017-12-02   9   
5989467  724718   667978  4827537    pv  1512140650  2017-12-01  23   
4360514  648388  4111010  3607361    pv  1512218938  2017-12-02  20   

                             time  
4324551 2017-11-27 19:46:07+08:00  
4688525 2017-12-02 16:31:13+08:00  
3939379 2017-11-27 21:29:10+08:00  
503

## 3.数据分析
### 3.1数据准备

In [43]:
#.用户行为概览
user_behav_df = Raw_data_df.groupby(['行为类型'])["用户ID"].nunique()
print(user_behav_df)

行为类型
buy     144026
cart    290074
fav     137794
pv      923653
Name: 用户ID, dtype: int64


In [44]:
#行为类型计数
count = Raw_data_df['行为类型'].value_counts()
print(count)

pv      7916321
cart     489929
fav      254229
buy      180290
Name: 行为类型, dtype: int64


In [45]:
#组合表
fin_df = pd.merge(user_behav_df,count,how= 'inner',left_index=True,right_index=True).sort_values("行为类型",inplace=False,ascending=False)
print(fin_df)

        用户ID     行为类型
pv    923653  7916321
cart  290074   489929
fav   137794   254229
buy   144026   180290


In [46]:
date_use_df = Raw_data_df.groupby(['时间'])["用户ID"].nunique().to_frame()
date_click_df = Raw_data_df.groupby(['时间']).size().to_frame()
date_use_df.columns = ["用户数"]
date_click_df.columns = ["点击数"]
print(date_use_df,date_click_df)

               用户数
时间                
2017-11-26  367919
2017-11-27  389328
2017-11-28  386720
2017-11-29  395207
2017-11-30  403297
2017-12-01  411626
2017-12-02  525039
2017-12-03  520207                 点击数
时间                 
2017-11-26   938738
2017-11-27  1009655
2017-11-28   988205
2017-11-29  1028563
2017-11-30  1045862
2017-12-01  1085502
2017-12-02  1377320
2017-12-03  1366924


In [47]:
# 分时段点击量和用户访问数
hour_df = Raw_data_df.groupby(['时段'])["用户ID"].nunique().to_frame().sort_index()
print(hour_df)

      用户ID
时段        
0   126766
1    61468
2    34450
3    24346
4    21433
5    28076
6    59802
7   108565
8   165941
9   200830
10  230392
11  228510
12  231121
13  244051
14  241764
15  248739
16  240738
17  226079
18  228235
19  266986
20  304037
21  330585
22  320759
23  245161


In [78]:
#选取点击量前十的商品类别
commodity_df = Raw_data_df.groupby(['商品类目ID'])["用户ID"].nunique().to_frame().sort_values(by = ["用户ID"],ascending = False).head(10)
commodity_df = commodity_df.reset_index()
print(commodity_df)
commodity_buy_df = Raw_data_df.loc[Raw_data_df["行为类型"] == "buy"].groupby(['商品类目ID'])["用户ID"].nunique().to_frame().sort_values(by = ["用户ID"],ascending = False).head(10)
commodity_buy_df = commodity_buy_df.reset_index()
print(commodity_buy_df)

    商品类目ID    用户ID
0  4756105  172381
1  4145813  142477
2   982926  140455
3  2355072  132484
4  3607361  122507
5  1320293  103931
6  4801426  103030
7  2520377   99086
8  3002561   79124
9  2465336   78721
    商品类目ID  用户ID
0  1464116  2903
1  2735466  2781
2  2885642  2596
3  4145813  2595
4  4756105  2303
5  4801426  2229
6   982926  2077
7  4159072  1542
8  2640118  1474
9  3002561  1409


### 3.2可视化图表
3.2.1 用户行为概览


In [48]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [49]:
fig = make_subplots(rows=1, cols=2, specs=[[{}, {'type':'domain'}]])
trace0 = go.Bar(
    x = fin_df.index,
    y = fin_df["行为类型"],
    orientation = 'v' , 
    name = "柱图",
    texttemplate='%{y}', 
    textposition="outside",
    hovertemplate = "%{x}: <br>共:%{y}次",
    )

trace1 = go.Pie(
    labels = fin_df.index,
    values = fin_df["行为类型"],
    name = "饼图",
    textinfo='label+percent',
    insidetextorientation='horizontal',
    hovertemplate = "%{label}共:%{value}次，占比%{percent}",
    hole = .4,
    #pull =[0.2, 0,0, 0]
    )
layout = dict(
    title='用户行为分布',
    legend={"x":1.05,"y":1.2,"font":dict(size=15),"bordercolor" : "grey","borderwidth": 1},  #设置图例标志的大小和位置
    xaxis = {"type":"category","title":"数量","showline":True},
    yaxis = {"title":"行为类型","showline":True},
    )
fig.append_trace(trace0,1,1)
fig.append_trace(trace1,1,2)
fig['layout'].update(layout)
fig.show()

In [50]:
3.2.2.用户活跃情况
观察用户9天内的活跃变化趋势，以及每天时间段内活跃变化趋势。

SyntaxError: invalid syntax (<ipython-input-50-e1223449a907>, line 1)

In [51]:
fig = go.Figure()
trace = go.Bar(
                    x = date_use_df.index,
                    y = date_use_df["用户数"],
                    orientation = 'v' , 
                    texttemplate = '%{y}', 
                    textposition = "outside",
                    name = '用户访问数',
                    hovertemplate = "%{x}: <br>共:%{y}次",
                    )
trace1 = go.Scatter(x = date_use_df.index, y = date_click_df["点击数"],
                    mode = 'lines+markers',
                    name = '点击量',
                    xaxis='x',
                    yaxis='y2',
                        )

data = [trace, trace1]
layout = go.Layout(
    yaxis2=dict(anchor='x', overlaying='y', side='right')#设置坐标轴的格式，一般次坐标轴在右侧
)
fig = go.Figure(data=data, layout=layout)
fig.show()

11月25日、26日，以及12月2日跟3日，这4天是周末，可以看到不论是从点击次数还是人数上，都要高于中间的5天。而12月初的周末PV量明显增加，可以考虑是否有12月初营销活动的影响。

In [55]:
fig = go.Figure()
trace = go.Bar(
                    x = hour_df.index,
                    y = hour_df["用户ID"],
                    orientation = 'v' , 
                    texttemplate = '%{y}', 
                    textposition = "outside",
                    name = '用户数',
                    hovertemplate = "%{x}: <br>共:%{y}次",
                    )
trace1 = go.Scatter(x = hour_df.index, y = hour_df["用户ID"],
                    mode = 'lines+markers',
                    name = '用户数',
                    xaxis='x',
                    yaxis='y2',
                        )

data = [trace, trace1]
layout = go.Layout(
    yaxis2=dict(anchor='x', overlaying='y', side='right')#设置坐标轴的格式，一般次坐标轴在右侧
)
fig = go.Figure(data=data, layout=layout)
fig.show()

在这9天的时间内，平均每天的浏览次数和人数，都在晚上21点左右达到峰值，且晚上19点-23点这段时间，是用户活跃高峰时段，可以考虑在这个时段，通过广告投放、微信QQ支付宝推广等方式，吸引更多客户。

In [94]:
fig = make_subplots(rows=1, cols=2, specs=[[{}, {'type':'domain'}]])
trace0 = go.Bar(
    x = commodity_df["商品类目ID"],
    y = commodity_df["用户ID"],
    orientation = 'v' , 
    name = "柱图",
    texttemplate='%{y}', 
    textposition="outside",
    hovertemplate = "%{x}: <br>共:%{y}次",
    )

trace1 = go.Pie(
    labels = commodity_df["商品类目ID"],
    values = commodity_df["用户ID"],
    name = "饼图",
    textinfo='label+percent',
    insidetextorientation='horizontal',
    hovertemplate = "%{label}共:%{value}次，占比%{percent}",
    hole = .4,
    #pull =[0.2, 0,0, 0]
    )
layout = dict(
    title='商品点击量top10',
    legend={"x":1.05,"y":1.2,"font":dict(size=15),"bordercolor" : "grey","borderwidth": 1},  #设置图例标志的大小和位置
    xaxis = {"type":"category","title":"商品类别","showline":True},
    yaxis = {"title":"点击量","showline":True},
    )
fig.append_trace(trace0,1,1)
fig.append_trace(trace1,1,2)
fig['layout'].update(layout)
fig.show()